In [ ]:
import urllib.request as ur

print("Automatic migration of ProActive catalog ...")

workflowCatalogURL = 'https://try.activeeon.com/workflow-catalog/'

wfURL = 'http://try.activeeon.com:8080/workflow-catalog/buckets/1/workflows/4/revisions/1?alt=xml'

dlf = open("test.xml", "b+w")
stream = ur.urlopen(wfURL)
dlf.write(stream.read())
dlf.close()

print("Finish Download ...")


In [ ]:
import json
import urllib.request as ur

meta = json.load(ur.urlopen("http://try.activeeon.com:8080/workflow-catalog/buckets/1/workflows/4"))
#print(meta)
#print(meta.get("_links").get("content").get("href"))
url = meta.get("_links").get("content").get("href")

In [ ]:
import json
import urllib.request as ur

#bucket meta data in the catalog(dict)
metaBucket = json.load(ur.urlopen("http://try.activeeon.com:8080/workflow-catalog/buckets"))
#bucket list(list)
bucketList = metaBucket.get("_embedded").get("bucketMetadataList")
for l in bucketList:
    wfurl = "http://try.activeeon.com:8080/workflow-catalog/buckets/"+str(l.get("id"))+"/workflows"
    #workflow meta data in the buckets (dict)
    metaWorkflow = json.load(ur.urlopen(wfurl))
    #worflow list(list)
    workflowList = metaWorkflow.get("_embedded").get("workflowMetadataList")
    for w in workflowList:
        print(w.get("name"))


In [ ]:
import json
import urllib.request as ur

bucketURL = "http://try.activeeon.com:8080/workflow-catalog/buckets/1/workflows/"

metaWorkflow = json.load(ur.urlopen(bucketURL))
workflowList = metaWorkflow.get("_embedded").get("workflowMetadataList")
for w in workflowList:
    wf = bucketURL + str(w.get("id"))
    wfmeta = json.load(ur.urlopen(wf))
    wfURL = wfmeta.get("_links").get("content").get("href")
    file_name = wfmeta.get("name")+".xml"
    u = ur.urlopen(wfURL)
    f = open(file_name, 'wb')
    f.write(u.read())
    f.close()
print("Finish Downloading ...")

In [ ]:
import os
import json
import urllib.request as ur
import zipfile
import shutil

bucket = "http://try.activeeon.com:8080/workflow-catalog/buckets/1"
bucketURL = bucket + "/workflows/"
meta = json.load(ur.urlopen(bucket))
name = meta.get("name")
mypath = "./"+name
if not os.path.isdir(mypath):
   os.makedirs(mypath)

metaWorkflow = json.load(ur.urlopen(bucketURL))
workflowList = metaWorkflow.get("_embedded").get("workflowMetadataList")
for w in workflowList:
    wf = bucketURL + str(w.get("id"))
    wfmeta = json.load(ur.urlopen(wf))
    wfURL = wfmeta.get("_links").get("content").get("href")
    file_name = wfmeta.get("name")+".xml"
    fullfilename = os.path.join(mypath, file_name)
    u = ur.urlopen(wfURL)
    f = open(fullfilename, 'wb')
    f.write(u.read())
    f.close()
print("Finish Downloading ...")

zipf = zipfile.ZipFile(name+".zip", 'w', zipfile.ZIP_DEFLATED)

for root, dirs, files in os.walk(mypath):
    for file in files:
        zipf.write(os.path.join(root, file))
print("Finish Zipping ...")

if os.path.exists(mypath):
    shutil.rmtree(mypath)
    
print("Original Folder removed ...")

In [ ]:
######################################
#ProActive Catalog Migration Download#
######################################

import os
import sys
import json
import getopt
import shutil
import zipfile
import urllib.request as ur

def downloadWFs(bucket):
    bucketURL = bucket + "/workflows/"
    meta = json.load(ur.urlopen(bucket))
    name = meta.get("name")
    mypath = "./"+name
    if not os.path.isdir(mypath):
        os.makedirs(mypath)
    metaWorkflow = json.load(ur.urlopen(bucketURL))
    workflowList = metaWorkflow.get("_embedded").get("workflowMetadataList")
    print("Begin Downloading Bucket: " +name+" ...")
    for w in workflowList:
        wf = bucketURL + str(w.get("id"))
        wfmeta = json.load(ur.urlopen(wf))
        wfURL = wfmeta.get("_links").get("content").get("href")
        file_name = wfmeta.get("name")+".xml"
        fullfilename = os.path.join(mypath, file_name)
        u = ur.urlopen(wfURL)
        f = open(fullfilename, 'wb')
        f.write(u.read())
        f.close()
    print("Finish Downloading Bucket: "+name+".")

def zipBucket(bucket):
    bucketURL = bucket + "/workflows/"
    meta = json.load(ur.urlopen(bucket))
    name = meta.get("name")
    mypath = "./"+name
    myname = mypath + ".zip"
    zipf = zipfile.ZipFile(myname, 'w', zipfile.ZIP_DEFLATED)
    print("Begin Zipping Bucket: " +name+" ...")
    for root, dirs, files in os.walk(mypath):
        for file in files:
            zipf.write(os.path.join(root, file))
    print("Finish Zipping Bucket: "+name+".")
    
    if os.path.exists(mypath):
        shutil.rmtree(mypath)
    
    return os.path.abspath(myname)

def migration(catalogURL, path):
    if path == "./":
        path = os.path.abspath(os.curdir) + "/CatalogWorkflows"
    else:
        path = path + "/CatalogWorkflows"
    if not os.path.isdir(path):
        os.makedirs(path)    
    bucketsURL = catalogURL+"/buckets"
    metaBucket = json.load(ur.urlopen(bucketsURL))
    bucketList = metaBucket.get("_embedded").get("bucketMetadataList")
    for l in bucketList:
        bucket = bucketsURL+"/"+str(l.get("id"))
        downloadWFs(bucket)
        name = zipBucket(bucket)
        shutil.move(name, path+"/"+name.split('/')[-1])
    print("Migration Finished ...")
    
    
if __name__ == "__main__":    
    catalogURL = "http://tryqa.activeeon.com:8080/workflow-catalog"
    path = "/Users/sophiesong/Desktop"
    #path = "./"
    migration(catalogURL, path)

In [ ]:
name = "/Users/sophiesong/Documents/workspace/activeeon/Examples.zip"
print(name.split('/')[-1])

In [29]:
import os
import json
import urllib.request as ur
import zipfile
import shutil

bucket = "https://tryqa.activeeon.com/catalog/buckets"
meta = json.load(ur.urlopen(bucket))
print(meta[-1].get('id'))

1007


In [39]:
####################################
#ProActive Catalog Migration Upload#
####################################

import os
import sys
import json
import getopt
import requests
import urllib.request as ur

#####Walk through a directory and Create a bucket for each zip file, then upload this zip to the new created bucket######

#rootDir = '/Users/sophiesong/Desktop/CatalogWorkflows/'
#newCatalog = 'https://tryqa.activeeon.com/catalog/'
def upload(rootDir, newCatalog):    
    buckets = newCatalog + 'buckets/'
    bucketURL = buckets+'?name='
    list_dirs = os.walk(rootDir)
    for root, dirs, files in list_dirs:
        for f in files:
            zipName = os.path.join(root, f).split('/')[-1]
            if(zipName.split('.')[-1] == 'zip'):
                bucketName =  zipName.split('.')[0]+'_0'
                response = requests.post(bucketURL+bucketName)
                meta = json.load(ur.urlopen(buckets))
                bucketID = meta[-1].get('id')
                #print(meta[-1].get('id'))
                url = buckets+str(bucketID)+'/resources?kind=workflow&commitMessage=migration&contentType=zip'
                files = {'file': open(rootDir + zipName, 'rb')}
                r = requests.post(url, files=files)
        

if __name__ == "__main__":    
    rootDir = '/Users/sophiesong/Desktop/CatalogWorkflows/'
    newCatalog = 'https://tryqa.activeeon.com/catalog/'
    upload(rootDir, newCatalog)

In [4]:
import requests

########Create a new bucket#########

url = "https://tryqa.activeeon.com/catalog/buckets?name="

response = requests.post(url+"test")


#curl -X GET --header 'Accept: */*' 'https://trydev.activeeon.com/catalog/buckets?name=Example'
#https://trydev.activeeon.com/catalog/buckets?name=test&owner=GROUP%3Apublic-objects
#curl -X POST --header 'Content-Type: application/json' --header 'Accept: */*' 'https://trydev.activeeon.com:8080/catalog/buckets?name=test&owner=GROUP%3Apublic-objects'



########Upload an archive to a bucket#########

# curl -X POST --header 'Content-Type: multipart/form-data' --header 'Accept: application/json' 'http://tryqa.activeeon.com:8080/catalog/buckets/1010/resources?kind=workflow&commitMessage=migration&contentType=zip' -F "file=@/Users/sophiesong/Desktop/CatalogWorkflows/Addons.zip"


url = 'http://tryqa.activeeon.com:8080/catalog/buckets/1010/resources?kind=workflow&commitMessage=migration&contentType=zip'
files = {'file': open('/Users/sophiesong/Desktop/CatalogWorkflows/Machine_Learning.zip', 'rb')}
r = requests.post(url, files=files)